In [1]:
plus1(x) = 1 + x 
methods(plus1)

# 1 method for generic function "plus1":
[1] plus1(x) in Main at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:1

In [2]:
module Foo 

using LinearAlgebra
plus1(X::AbstractMatrix) = I + X

end

methods(plus1)

# 1 method for generic function "plus1":
[1] plus1(x) in Main at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:1

In [3]:
using .Foo
methods(plus1)

# 1 method for generic function "plus1":
[1] plus1(x) in Main at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:1

In [4]:
methods(Foo.plus1)

# 1 method for generic function "plus1":
[1] plus1(X::AbstractMatrix) in Main.Foo at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:4

In [12]:
module Bar 

export plus1
plus1(t::Tuple) = (1,t...)

export plusone
plusone(t::Tuple) = plus1(t)

end

using .Bar
methods(plus1)

# 2 methods for generic function "plus1":
[1] plus1(v::AbstractVector) in Main.Baz at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:4
[2] plus1(x) in Main at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:1

In [14]:
plus1((2,3))

MethodError: MethodError: no method matching +(::Int64, ::Tuple{Int64, Int64})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at /Applications/julia-rosetta/Julia-1.7.app/Contents/Resources/julia/share/julia/base/operators.jl:655
  +(::T, !Matched::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at /Applications/julia-rosetta/Julia-1.7.app/Contents/Resources/julia/share/julia/base/int.jl:87
  +(::Union{Int16, Int32, Int64, Int8}, !Matched::BigInt) at /Applications/julia-rosetta/Julia-1.7.app/Contents/Resources/julia/share/julia/base/gmp.jl:535
  ...

In [7]:
Bar.plus1((2,3))

(1, 2, 3)

In [13]:
plusone((2,3))

(1, 2, 3)

In [8]:
module Baz

import Main:plus1 
plus1(v::AbstractVector) = [1;v]

end

using .Baz 
methods(plus1)

# 2 methods for generic function "plus1":
[1] plus1(v::AbstractVector) in Main.Baz at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:4
[2] plus1(x) in Main at /Users/driscoll/repos/julia-workshop/modules_packages.ipynb:1

In [9]:
plus1(pi)

4.141592653589793

In [10]:
plus1([2,3,4])

4-element Vector{Int64}:
 1
 2
 3
 4